In [2]:
from gensim.test.utils import datapath
from gensim.models import FastText
import pandas as pd
import nltk
from gensim.parsing.preprocessing import STOPWORDS
import re
nltk.download('stopwords') 
nltk.download('wordnet') 

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
STOPWORDS

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [4]:
data_tip=pd.read_json('/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json', lines=True)

In [5]:

tip_text = data_tip['text']

In [6]:
tip_text[:3]

0                       Avengers time with the ladies.
1    They have lots of good deserts and tasty cuban...
2               It's open even when you think it isn't
Name: text, dtype: object

In [ ]:
!python -m nltk.downloader -d /usr/local/share/nltk_data all

In [ ]:
import spacy

def process_text(document,stopwords=STOPWORDS):
    nlp = spacy.load("en_core_web_sm")
    document = re.sub(r'\s+', ' ', document, flags=re.I) 
    document = re.sub(r'\W', ' ', str(document)) 
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document) 
    document = document.lower() 
    tokens = document.split()
    doc_spacy=nlp(document)
    
    lemma_txt = ["".join(token.lemma_) for token in doc_spacy]
    lemma_no_stop_txt = [word for word in lemma_txt if word not in STOPWORDS]
    tokens = [word for word in tokens if len(word) > 3]
    clean_txt = ' '.join(lemma_no_stop_txt)
    return clean_txt

tip_text_preprocessed = tip_text.apply(process_text)

model3 = FastText(vector_size=4, workers=7, window=7, min_count=1)
model3.build_vocab(corpus_iterable=tip_text_preprocessed)
total_words = model3.corpus_total_words
model3.train(corpus_iterable=tip_text_preprocessed, total_words=total_words, epochs=5)


In [ ]:
import cupy as cp
nlp = spacy.load("en_core_web_sm")

# Load stopwords (if not already loaded)
STOPWORDS = spacy.lang.en.stop_words.STOP_WORDS

def process_text(document, stopwords=STOPWORDS):
    # Regular expression preprocessing
    document = re.sub(r'\s+', ' ', document, flags=re.I) 
    document = re.sub(r'\W', ' ', str(document)) 
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document) 
    document = document.lower() 
    
    # Tokenization using SpaCy
    doc_spacy = nlp(document)
    
    # Lemmatization and stopword removal
    lemma_no_stop_txt = [token.lemma_ for token in doc_spacy if token.lemma_ not in stopwords]
    
    # Filtering tokens based on length
    tokens = [word for word in lemma_no_stop_txt if len(word) > 3]
    
    # Joining tokens into a clean text
    clean_txt = ' '.join(tokens)
    return clean_txt

# Assuming tip_text is a pandas Series containing the text data
tip_text_preprocessed = tip_text.apply(process_text)

# Transfer data to GPU
tip_text_preprocessed_gpu = cp.asarray(tip_text_preprocessed)

# Initialize FastText model with GPU support
model3 = FastText(vector_size=4, workers=7, window=7, min_count=1)

# Build vocabulary on GPU
model3.build_vocab(corpus_iterable=tip_text_preprocessed_gpu)

# Train FastText model on GPU
total_words = model3.corpus_total_words
model3.train(corpus_iterable=tip_text_preprocessed_gpu, total_words=total_words, epochs=5)

In [ ]:
nltk.download()

In [ ]:
len(model3.wv.key_to_index)

In [ ]:
sims = model3.wv.most_similar('lots',topn=10) 

In [ ]:
sims